In [23]:
import cirq
from cirq.aqt.aqt_device import get_aqt_device
from cirq.aqt.aqt_sampler import AQTSampler
access_token = '2633b2d14dae4d9da7c9df2503857334'

# define the length of the grid.
length = 3
# define qubits on the grid.
device, qubits = get_aqt_device(length**2)
print(qubits)

[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5), cirq.LineQubit(6), cirq.LineQubit(7), cirq.LineQubit(8)]
2


In [20]:
circuit = cirq.Circuit()
circuit.append([cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0],
               strategy=cirq.InsertStrategy.EARLIEST)
circuit.append([cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1],
               strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
print(circuit)

0: ───H───

1: ───H───

2: ───H───

3: ───H───

4: ───H───

5: ───H───

6: ───H───

7: ───H───

8: ───H───


In [3]:
for i, m in enumerate(circuit):
    print('Moment {}: {}'.format(i, m))

Moment 0: H((0, 0)) and H((0, 2)) and H((1, 1)) and H((2, 0)) and H((2, 2))
Moment 1: X((0, 1)) and X((1, 0)) and X((1, 2)) and X((2, 1))


In [4]:
def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square grid of given length."""
    rot = cirq.XPowGate(exponent=half_turns)
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)

(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


In [5]:
import random
def rand2d(rows, cols):
    return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length - 1, length)
    # links within a column
    jc = rand2d(length, length - 1)
    return (h, jr, jc)

h, jr, jc = random_instance(length)
print('transverse fields: {}'.format(h))
print('row j fields: {}'.format(jr))
print('column j fields: {}'.format(jc))

transverse fields: [[1, -1, -1], [-1, -1, 1], [1, -1, 1]]
row j fields: [[-1, -1, -1], [1, -1, 1]]
column j fields: [[1, 1], [-1, 1], [1, 1]]


In [6]:
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

In [7]:
def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i + 1, j))
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i + 1, j))
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i + 1, j))

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j + 1))
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i, j + 1))
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j + 1))

In [8]:
def one_step(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
    length = len(h)
    yield rot_x_layer(length, x_half_turns)
    yield rot_z_layer(h, h_half_turns)
    yield rot_11_layer(jr, jc, j_half_turns)

h, jr, jc = random_instance(length)

circuit = cirq.Circuit()    
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3),
               strategy=cirq.InsertStrategy.EARLIEST)
print(circuit)

                           ┌──────────┐   ┌──────────┐
(0, 0): ───X^0.1────────────@────────────────────────────────@───────────────────────────────────
                            │                                │
(0, 1): ───X^0.1───Z^0.2────┼────X─────────@─────────────X───@^0.3───@───────────────────────────
                            │              │                         │
(0, 2): ───X^0.1───Z^0.2────┼────@─────────┼─────────────────────────@^0.3───────────────────────
                            │    │         │
(1, 0): ───X^0.1───Z^0.2────@^0.3┼─────────┼────@────────X───────────────────@───────X───────────
                                 │         │    │                            │
(1, 1): ───X^0.1───X─────────────┼─────────@^0.3┼────────X───@───────X───────@^0.3───X───@───────
                                 │              │            │                           │
(1, 2): ───X^0.1─────────────────@^0.3─────@────┼────────────┼───────────────────────────@^0.3───
        

In [9]:
simulator = cirq.Simulator()
circuit = cirq.Circuit()
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))
results = simulator.run(circuit, repetitions=100)
print(results.histogram(key='x'))

Counter({0: 73, 256: 6, 32: 4, 8: 4, 1: 3, 16: 3, 64: 2, 2: 2, 4: 2, 128: 1})


In [10]:
import numpy as np

def energy_func(length, h, jr, jc):
    def energy(measurements):
        # Reshape measurement into array that matches grid shape.
        meas_list_of_lists = [measurements[i * length:(i + 1) * length]
                              for i in range(length)]
        # Convert true/false to +1/-1.
        pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

        tot_energy = np.sum(pm_meas * h)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
        return tot_energy
    return energy
print(results.histogram(key='x', fold_func=energy_func(length, h, jr, jc)))

Counter({5: 73, 1: 11, 3: 11, 7: 3, -1: 2})


In [11]:
def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(length, h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions
print('Value of the objective function {}'.format(obj_func(results)))

Value of the objective function 4.28


In [12]:
import sympy
circuit = cirq.Circuit()
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
circuit.append(one_step(h, jr, jc, alpha, beta, gamma))
circuit.append(cirq.measure(*qubits, key='x'))
print(circuit)

                              ┌──────────────┐   ┌──────────────┐
(0, 0): ───X^alpha─────────────@────────────────────────────────────────@───────────────────────────────────────────M('x')───
                               │                                        │                                           │
(0, 1): ───X^alpha───Z^beta────┼──────X───────────@─────────────────X───@^gamma───@─────────────────────────────────M────────
                               │                  │                               │                                 │
(0, 2): ───X^alpha───Z^beta────┼──────@───────────┼───────────────────────────────@^gamma───────────────────────────M────────
                               │      │           │                                                                 │
(1, 0): ───X^alpha───Z^beta────@^gamma┼───────────┼──────@──────────X───────────────────────@─────────X─────────────M────────
                                      │           │      │                  

In [13]:
sweep = (cirq.Linspace(key='alpha', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='beta', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='gamma', start=0.1, stop=0.9, length=5))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
for result in results:
    print(result.params.param_dict, obj_func(result))

OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.1)]) 4.48
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.30000000000000004)]) 4.56
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.5)]) 4.38
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.7000000000000001)]) 4.26
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.9)]) 4.88
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.1)]) 4.36
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.30000000000000004)]) 4.62
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.5)]) 4.24
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.7000000000000001)]) 4.5
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.9)]) 4.48
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.1)]) 4.4
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.30000000000000004)]) 4.52
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma'

OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.1)]) 10.32
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.30000000000000004)]) 10.0
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.5)]) 10.3
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.7000000000000001)]) 9.94
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.9)]) 9.9
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.1)]) 10.12
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.30000000000000004)]) 10.04
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.5)]) 10.22
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.7000000000000001)]) 9.68
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.9)]) 10.54
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.1)]) 10.02
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.30000000000000004)]) 9.78
OrderedDict([('

In [14]:
sweep_size = 10
sweep = (cirq.Linspace(key='alpha', start=0.0, stop=1.0, length=10)
         * cirq.Linspace(key='beta', start=0.0, stop=1.0, length=10)
         * cirq.Linspace(key='gamma', start=0.0, stop=1.0, length=10))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)

min = None
min_params = None
for result in results:
    value = obj_func(result)
    if min is None or value < min:
        min = value
        min_params = result.params
print('Minimum objective value is {}.'.format(min))

Minimum objective value is -1.42.
